# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
import findspark
findspark.init('/home/cheng/spark-2.3.0-bin-hadoop2.7')
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('recommender').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [5]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
data.head(5)

[Row(movieId=2, rating=3.0, userId=0),
 Row(movieId=3, rating=1.0, userId=0),
 Row(movieId=5, rating=2.0, userId=0),
 Row(movieId=9, rating=4.0, userId=0),
 Row(movieId=11, rating=1.0, userId=0)]

In [24]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [7]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   3.0|    14| 1.9262125|
|     31|   1.0|     0|0.39283374|
|     85|   1.0|    13| 2.5892138|
|     85|   5.0|    16|  3.197308|
|     85|   1.0|     4| 2.7494435|
|     85|   1.0|    29| 0.7957745|
|     65|   1.0|    22| 1.4205308|
|     65|   5.0|    23| 0.8468536|
|     65|   1.0|    24| 1.6881912|
|     53|   3.0|    13|  3.194272|
|     53|   5.0|     8| 1.5942744|
|     78|   1.0|    28| 0.6983751|
|     78|   1.0|    20| 0.9239678|
|     34|   1.0|    16| 1.1114438|
|     34|   1.0|    17|-0.5271191|
|     34|   3.0|    25| 2.9914339|
|     34|   1.0|    14| 1.1243055|
|     81|   1.0|     1|  2.235907|
|     81|   2.0|     9|0.59092176|
|     28|   1.0|    17| 1.4417567|
+-------+------+------+----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.472256925585813


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     21|    11|
|     25|    11|
|     41|    11|
|     45|    11|
|     48|    11|
|     70|    11|
|     77|    11|
|     89|    11|
|     90|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)

In [15]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     90|    11| 5.0849376|
|     41|    11| 3.0201437|
|     70|    11|   3.00882|
|     77|    11| 2.6555681|
|     89|    11| 2.5493517|
|     45|    11| 1.7675389|
|     48|    11| 1.1675305|
|     13|    11|0.81205916|
|     25|    11| 0.7200771|
|     21|    11|-1.5605965|
+-------+------+----------+



In [16]:
test.filter(test['userId']==11).show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|     13|   4.0|    11|
|     21|   1.0|    11|
|     25|   1.0|    11|
|     41|   1.0|    11|
|     45|   1.0|    11|
|     48|   5.0|    11|
|     70|   1.0|    11|
|     77|   1.0|    11|
|     89|   1.0|    11|
|     90|   4.0|    11|
+-------+------+------+



# Great Job!